In [13]:
from sklearn.model_selection import train_test_split
from sec_certs.dataset.common_criteria import CCDataset
import json

In [14]:
cc_dset: CCDataset = CCDataset.from_json('/Users/adam/phd/projects/certificates/datasets/cc_full_dataset/CommonCriteria_dataset.json')

cve_rich_certs = [x for x in cc_dset if x.heuristics.related_cves]
cve_free_certs = [x for x in cc_dset if not x.heuristics.related_cves]

x_valid_cve_rich, x_test_cve_rich = train_test_split(cve_rich_certs, test_size=0.5)
x_valid_cve_free, x_test_cve_free = train_test_split(cve_free_certs, test_size=0.5)

validation_set = x_valid_cve_rich + x_valid_cve_free
test_set = x_test_cve_rich + x_test_cve_free

validation_set = {x.dgst: x.name for x in validation_set}
test_set = {x.dgst: x.name for x in test_set}

In [15]:
with open('./validation_set.json', 'w') as handle:
    json.dump(validation_set, handle, indent=4)

with open('./test_set.json', 'w') as handle:
    json.dump(test_set, handle, indent=4)